## get_portfolio_var - 
1. Use the barchart api to retrieve stock historical data;
2. Create a correlation matrix from the historical data; 
3. Get standard deviations from the historical data;
4. Create long/short position information for each security retrieved
5. Create a portolio VaR using all of the above

## Imports

In [100]:
import pandas as pd
import numpy as np
import os
import sys
import datetime
from os.path import expanduser
from scipy.stats import norm
from numpy import linalg as LA

### Add to sys.path
Within the barchart_api module, references like: ```from barchartapi import barchart_api``` will not work unless you append to sys.path the following folders:
1. project working directory (which holds barchart_api.py),
2. project directory,
3. workspace directory.

In [101]:
WORKSPACE_DIR = '../..' 
PROJECT_DIR = f'{WORKSPACE_DIR}/barchartapi' 
WORKING_DIR = f'{PROJECT_DIR}/barchartapi' 
if WORKING_DIR not in sys.path:
    sys.path.append(WORKING_DIR)
if PROJECT_DIR not in sys.path:
    sys.path.append(PROJECT_DIR)
if WORKSPACE_DIR not in sys.path:
    sys.path.append(WORKSPACE_DIR)
import barchart_api as bcapi

___
### Create paths to where you will store the history data

In [102]:
STOCKS_DIR = f'./temp_folder/stocks'
try:
    os.makedirs(STOCKS_DIR)
except:
    pass

___
### Create an instance of BcHist
1. must set bar_type to minutes, hour or daily
2. must set interval
 * for minutes, set interval to 1, 5, 15, 30 or 60
 * for other types, see https://www.barchart.com/ondemand/api/getHistory

In [103]:
# set this to 'free' or 'paid'
endpoint = 'free' # free or paid

# set the bar_type and the interval
bar_type='daily' # minutes, daily, monthly
interval=1 # 1,5,15,30,60

# create an instance 
api_key = open(f'./temp_folder/{endpoint}_api_key.txt','r').read()
endpoint_type=f'{endpoint}_url'
bch = bcapi.BcHist(api_key, bar_type=bar_type, interval=interval,endpoint_type = endpoint_type)


Barchart OnDemand Client: https://marketdata.websol.barchart.com/


### Set delete_old_csv_file to True if you want to re-retrieve the csv files that we use in the analysis

In [104]:
# true will cause barchartapi to refetch csv data, 
#   false will cause it to bypass any short_name that already has a csv file in the path sn_path below
delete_old_csv_file = False 

### Define a method to read your csv portfolio file

In [105]:
# define a method to retrieve the portfolio
def get_portfolio(csv_path=None):
    '''
    fetch a portfolio to use below
    '''
    p = csv_path
    if p is None:
        p = './hi_volume_stocks.csv'
    df_p = pd.read_csv(p)
    df_p['under'] = df_p.symbol.apply(lambda s: s.split("_")[0])    
    pnames = list(df_p.under)
    return (df_p,pnames)

### Get your portfolio
1. Enter **portfolio_folder_to_search** : a folder to search for a portfolio csv
2. Enter **portfolio_csv_name** : a csv file name in that folder that has 2 columns:
 * symbol column
 * position column
 *   

3. (default is ./hi_volume_stocks.csv)

In [106]:
# enter a folder and a name
portfolio_folder_to_search = '.'
portfolio_csv_name = 'fxe_portfolio.csv'

csv_file = portfolio_folder_to_search + "/" + portfolio_csv_name
print('possible csvs: ',list(filter(lambda d: '.csv' in str(d).lower(),os.listdir(portfolio_folder_to_search))))
p_tuple = get_portfolio(csv_file)
df_portfolio = p_tuple[0]
print('names in portfolio',p_tuple[1])

possible csvs:  ['fxe_portfolio.csv', 'hi_volume_stocks.csv']
names in portfolio ['FXE', 'XLE', 'XLU', 'SOYB', 'CORN', 'ZCK19', 'ZSK19', 'ZWK19', 'WEAT']


___
### Main Loop to fetch history data from Barchart

#### Inputs
On each call to BcHist.get_history, provide:
 * a beginning yyyymmdd integer
 * an ending yyyymmdd integer
 * a short name like:
  * a stock symbol like: SPY,USO,IBM, etc
  * a futures symbol like: CLJ18, GCG19, ESH17, etc

#### Outputs
The get_history method will return a tuple, where:
 * tup[0] provides a status

* tup[1] provides a pandas DataFrame of data, or None

In [107]:
#  set a date range
days_to_fetch = 120
dt_end = datetime.datetime.now()
dt_beg = dt_end - datetime.timedelta(days_to_fetch)
beg_yyyymmdd = '%04d%02d%02d' %(dt_beg.year,dt_beg.month,dt_beg.day)#20181201
end_yyyymmdd = '%04d%02d%02d' %(dt_end.year,dt_end.month,dt_end.day)#20190219


# set up other ETF names that can be used as a hedge to the portfolio
spydr_short_names = ['XLE','XLU','XLK','XLB','XLP','XLY','XLI','XLC','XLV','XLF']
equity_etf_names = ['DIA','SPY','QQQ']
commodity_etf_short_names = ['USO','UNG','DBC','DBA','GLD','USCI']
currency_etf_short_names = ['FXY','FXE','FXB','FXF','FXC','FXA']
all_names = spydr_short_names + equity_etf_names + commodity_etf_short_names + currency_etf_short_names + commodity_etf_short_names

# get the portfolio
port_names = p_tuple[1]

# set short_names to the desired list to get
short_names = list(set(
    port_names + spydr_short_names + ['SPY']))

for short_name in short_names:
    sn_path = f'{STOCKS_DIR}/{short_name}.csv'
    if os.path.isfile(sn_path):
        if delete_old_csv_file:
            os.remove(sn_path)
        else:
            print(f'BYPASSING: {short_name}')
            continue
    print(f'get_history: {short_name} BEGIN {datetime.datetime.now()}')
    tup = bch.get_history(short_name, beg_yyyymmdd, end_yyyymmdd)
    print(f'get_history: {short_name} WRITING DATA {datetime.datetime.now()}')
    tup[1].to_csv(sn_path,index=False)
    print(f'get_history: {short_name} END {datetime.datetime.now()}')
    

BYPASSING: XLE
BYPASSING: XLY
BYPASSING: XLU
BYPASSING: ZWK19
BYPASSING: SPY
BYPASSING: XLB
BYPASSING: XLC
BYPASSING: WEAT
BYPASSING: XLK
BYPASSING: CORN
BYPASSING: FXE
BYPASSING: XLI
BYPASSING: XLF
BYPASSING: SOYB
BYPASSING: XLV
BYPASSING: ZSK19
BYPASSING: ZCK19
BYPASSING: XLP


___
### Do something interesting with the returned data, like create a portfolio VaR

### First create correlations

In [108]:
days_of_std = 60

In [111]:

df_hist = None
for short_name in short_names:
    sn_path = f'{STOCKS_DIR}/{short_name}.csv'
    df_temp = pd.read_csv(sn_path).iloc[-1*days_of_std:]
    df_temp = df_temp[['tradingDay','close']]
    df_temp = df_temp.rename(columns={'close':f'{short_name}'})
    if df_hist is None:
        df_hist = df_temp.copy()
    else:
        df_hist = df_hist.merge(df_temp,how='inner',on='tradingDay')
df_corr = df_hist[df_portfolio.under].corr()
df_corr

,FXE,XLE,XLU,SOYB,CORN,ZCK19,ZSK19,ZWK19,WEAT
FXE,1.000000,-0.458799,-0.710159,0.398200,0.626913,0.632678,0.431093,0.612013,0.634432
XLE,-0.458799,1.000000,0.760448,0.084539,-0.396282,-0.447145,0.008208,-0.558690,-0.584786
XLU,-0.710159,0.760448,1.000000,-0.349888,-0.750790,-0.767971,-0.391091,-0.823662,-0.848706
SOYB,0.398200,0.084539,-0.349888,1.000000,0.793059,0.768758,0.988686,0.640819,0.623366
CORN,0.626913,-0.396282,-0.750790,0.793059,1.000000,0.993720,0.820021,0.926485,0.922051
ZCK19,0.632678,-0.447145,-0.767971,0.768758,0.993720,1.000000,0.802482,0.935966,0.932186
ZSK19,0.431093,0.008208,-0.391091,0.988686,0.820021,0.802482,1.000000,0.675979,0.658600
ZWK19,0.612013,-0.558690,-0.823662,0.640819,0.926485,0.935966,0.675979,1.000000,0.998284
WEAT,0.634432,-0.584786,-0.848706,0.623366,0.922051,0.932186,0.658600,0.998284,1.000000


### Next create standard deviations

In [112]:
cols = list(set(list(df_hist.columns.values))-set(['tradingDay']))
bars_per_day = 1
if bar_type.lower()!='daily':
    if bar_type.lower()=='minutes':
        bars_per_day = 8*2
perc_of_day = 1/bars_per_day
perc_of_year = perc_of_day/256
std_series = df_hist[cols].pct_change().iloc[1:].std()/perc_of_year**.5
df_std = pd.DataFrame({'stdev':list(std_series.values),'symbol':list(std_series.index.values)})
df_std.sort_values('symbol')

,stdev,symbol
14,0.113547,CORN
7,0.068828,FXE
15,0.117570,SOYB
4,0.187820,SPY
6,0.210797,WEAT
5,0.207811,XLB
12,0.240218,XLC
0,0.244194,XLE
11,0.182722,XLF
8,0.213324,XLI


#### Enter Positions

In [113]:
df_positions = df_portfolio[['under','position']].rename(columns={'under':'symbol'})
df_positions

,symbol,position
0,FXE,1060
1,XLE,250
2,XLU,-250
3,SOYB,1
4,CORN,1
5,ZCK19,50
6,ZSK19,50
7,ZWK19,50
8,WEAT,1


#### Get current prices

In [114]:
cols = list(filter(lambda c:'time' not in c,df_hist.columns.values))
vals = df_hist.iloc[-1:][cols].as_matrix().reshape(-1)
df_prices = pd.DataFrame({'symbol':cols,'price':vals})[['symbol','price']]
df_prices

,symbol,price
0,tradingDay,2019-03-15
1,XLE,65.51
2,XLY,110.6
3,XLU,58.19
4,ZWK19,462.25
5,SPY,281.31
6,XLB,55.4
7,XLC,47.02
8,WEAT,5.35
9,XLK,72.99


#### Merge position, prices, stds

In [115]:
df_positions_2 = df_positions.merge(df_prices,how='inner',on='symbol')
df_positions_3 = df_positions_2.merge(df_std,how='inner',on='symbol')
df_positions_3

,symbol,position,price,stdev
0,FXE,1060,108.04,0.068828
1,XLE,250,65.51,0.244194
2,XLU,-250,58.19,0.149043
3,SOYB,1,16.1501,0.117570
4,CORN,1,15.7493,0.113547
5,ZCK19,50,373.25,0.134818
6,ZSK19,50,909.25,0.124388
7,ZWK19,50,462.25,0.239725
8,WEAT,1,5.35,0.210797


#### Create position VaR's

In [116]:
VAR_CONFIDENCE = .99

In [117]:
df_positions_3['unit_var'] = df_positions_3.apply(lambda r: r.price * r.stdev * norm.ppf(VAR_CONFIDENCE) * (1/256)**.5 / r.price,axis=1 )
df_positions_3['position_var'] = df_positions_3.apply(lambda r: r.unit_var * r.position * r.price ,axis=1 )
df_positions_3

,symbol,position,price,stdev,unit_var,position_var
0,FXE,1060,108.04,0.068828,0.010007,1146.073610
1,XLE,250,65.51,0.244194,0.035505,581.482235
2,XLU,-250,58.19,0.149043,0.021670,-315.250495
3,SOYB,1,16.1501,0.117570,0.017094,0.276075
4,CORN,1,15.7493,0.113547,0.016509,0.260010
5,ZCK19,50,373.25,0.134818,0.019602,365.823717
6,ZSK19,50,909.25,0.124388,0.018086,822.215907
7,ZWK19,50,462.25,0.239725,0.034855,805.592278
8,WEAT,1,5.35,0.210797,0.030649,0.163973


#### Create portfolio VaR

In [118]:
# create an spy standard deviation that is the historical average
var_days = 1
spy_usual_daily_std = .16 
port_std = (df_positions_3.position_var.astype(float).as_matrix().T @ df_corr.astype(float).as_matrix() @ df_positions_3.position_var.astype(float).as_matrix())**.5
port_var = port_std * var_days**.5  
spy_curr_unit_var = float(df_std[df_std.symbol=='SPY'].stdev) * (var_days/256)**.5  * norm.ppf(VAR_CONFIDENCE) 
sp_dollar_equiv = port_var / spy_curr_unit_var 



In [119]:
print(f'portolio VaR: {round(port_var,2)}')
print(f"sp {var_days} day{'s' if var_days>1 else ''} unit VaR: {round(spy_curr_unit_var,2)}")
print(f'Equivalent S&P position (in dollars): {round(sp_dollar_equiv,2)}')


portolio VaR: 2707.66
sp 1 day unit VaR: 0.03
Equivalent S&P position (in dollars): 99151.16


### Do High Low Analysis

In [121]:
import pandas_datareader.data as pdr
t2 = datetime.datetime.now()
t1 = t2 - datetime.timedelta(120)
d1 = pdr.DataReader('CORN', 'yahoo', t1, t2)
d2 = pd.read_csv(f'{STOCKS_DIR}/CORN.csv').iloc[-1*days_of_std:].tail()

In [122]:
d1.tail()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2019-03-12,15.59,15.40,15.40,15.56,28400,15.56
2019-03-13,15.67,15.49,15.52,15.58,45200,15.58
2019-03-14,15.75,15.60,15.63,15.67,23100,15.67
2019-03-15,15.78,15.69,15.69,15.75,17600,15.75
2019-03-18,15.78,15.68,15.69,15.71,19969,15.71


In [123]:
d2.tail()

,close,high,low,open,openInterest,symbol,timestamp,tradingDay,volume
74,15.4000,15.5100,15.38,15.51,NaN,CORN,2019-03-11T00:00:00-04:00,2019-03-11,31800
75,15.5600,15.5912,15.40,15.40,NaN,CORN,2019-03-12T00:00:00-04:00,2019-03-12,28300
76,15.5812,15.6700,15.49,15.52,NaN,CORN,2019-03-13T00:00:00-04:00,2019-03-13,45200
77,15.6716,15.7499,15.60,15.63,NaN,CORN,2019-03-14T00:00:00-04:00,2019-03-14,23000
78,15.7493,15.7800,15.69,15.69,NaN,CORN,2019-03-15T00:00:00-04:00,2019-03-15,17600


In [136]:
df3 = d2.copy()
df3.index = df3.tradingDay.apply(lambda d: pd.Timestamp(d))
df3.index.name = 'Date'
newcols = {c:c[0].upper()+c[1:] for c in df3.columns.values}
df3 = df3.rename(columns=newcols)
df3 = df3[['High','Low','Open','Close','Volume']]
df3['Adj Close'] = df3.Close
df3.tail()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2019-03-11,15.5100,15.38,15.51,15.4000,31800,15.4000
2019-03-12,15.5912,15.40,15.40,15.5600,28300,15.5600
2019-03-13,15.6700,15.49,15.52,15.5812,45200,15.5812
2019-03-14,15.7499,15.60,15.63,15.6716,23000,15.6716
2019-03-15,15.7800,15.69,15.69,15.7493,17600,15.7493


In [139]:
list(df3.index)[0],list(d1.index)[0]

(Timestamp('2019-03-11 00:00:00'), Timestamp('2018-11-19 00:00:00'))

## END